In [1]:
# | default_exp utils.factory

In [2]:
# | exporti
from dataclasses import dataclass, field
from typing import Any, Callable, List
from functools import wraps

import httpx

import domolibrary_extensions.client as dec

In [3]:
# | hide
from nbdev.showdoc import show_doc
import pandas as pd
import asyncio

from pprint import pprint

# Factory Functions

To simplify implementing work chains, a factory function should issue API requests and capture actions taken including errors encountered. <br>

The function can be identified by and wrapped in a `factory_function` wrapper class to ensure appropriate inputs and exports are sent to the class. <br>

A factory function will always receive a `FactoryLog` and return a `ResponseFactory` class

## Factory Messages log activity within a function

In [4]:
# | exports


@dataclass
class FactoryMessage:
    stage: str  # description of the stage of a process
    message: str = "init"  # outcome
    stage_num: int = None
    is_success: bool = False

    """class for logging a stage of a process"""

    def to_json(self):
        return {key: value for key, value in self.__dict__.items() if value is not None}

    def to_string(self):
        string = " | ".join(
            [
                f"{key} - {str(value)}"
                for key, value in self.__dict__.items()
                if value is not None
            ]
        )

        if not self.is_success:
            string = f"💀 {string}"
        return string

    def __eq__(self, other):
        if not isinstance(other, FactoryMessage):
            return False
        return self.stage == other.stage

### sample flow using Factory Message
in this example `res` accumulates `FactoryMessage` throughout `foo` execution.

Because `res` is a list it is pass by reference into `foo` and therefore mutable within the function <br>
Similarly, `FactoryMessage` is also mutable, and therefore can be modified after instantiation.  Notice that `message` is updated even after it has passed into `res` 

In [5]:
res = []


def foo(res: List[FactoryMessage]):

    message = FactoryMessage(stage="foo")
    res.append(message)

    # do something
    message.message = "something happened"
    message.is_success = True

    return res


foo(res)

[FactoryMessage(stage='foo', message='something happened', stage_num=None, is_success=True)]

## Factory Response accumulate logs within a function

`FactoryResponse` uses attribute `messages`, a list, to accumulate `FactoryMessage`

In a larger implementation with multiple factory_functions, each factory_function should return a `FactoryResponse`, and then a `Log` should unite all the `FactoryResponse`.

In [6]:
# | exports


@dataclass
class FactoryResponse:
    function_name: str
    id: str  # identify a set of log entries

    messages: List[FactoryMessage] = field(
        default_factory=lambda: []
    )  # capture intermediate steps of the factory_function
    response: Any = field(
        repr=False, default=None
    )  # final object to return from the factory_function
    is_success: bool = False
    location = None

    """Response class for handling logging of a factory function.
    Accumulates messages as the factory unfolds
    """

    def to_json(self) -> List[dict]:
        columns = [
            "location",
            "function_name",
            "stage_num",
            "stage",
            "id",
            "is_success",
            "message",
            "response",
        ]
        s = [{**self.__dict__, **msg.to_json()} for msg in self.messages]

        return [{col: obj[col] for col in columns if obj.get(col)} for obj in s]

    def add_message(self, message: FactoryMessage):
        if message in self.messages:
            print("message in messages", message)

        self.messages.append(message)

    def test_success(self):
        """tests if all factory_messages are successful"""
        self.is_success = all([message.is_success for message in self.messages])
        return self.is_success

    def __eq__(self, other):
        if not isinstance(other, FactoryResponse):
            return False

        return self.id == other.id and self.function_name == other.function_name

### sample flow of a function using Factory Response

1. factory_function should receive an instantiated `FactoryResponse` 
2. instantiate `FactoryMessage` to track outcomes of API requests and accumulate logs
3. set `res.response` to an appropriate output
4. update `FactoryResponse` by testing for message success with `res.test_success()`

In [7]:
def foo(res: FactoryResponse) -> FactoryResponse:

    # stage1 = 'hello world'
    message = FactoryMessage(stage="hello_world")

    # add message to res.messages to accumulate logs
    res.add_message(message)

    try:
        # do something
        message.is_success = True
        message.message = "something happend"

    except Exception as e:
        message.is_success = False
        message.message = e

    # update response
    res.response = "we did the thing"

    # validate that all messages in res.messages report is_success
    res.test_success()

    return res


logs = []  # will accumulate FactoryResponses

for id in ["test@me.com", "test2@me.com"]:  # run foo with each id
    res = FactoryResponse(function_name=foo.__name__, id=id)

    foo(res=res)
    logs.append(res)

    pprint(res)
    print("\n")


pd.DataFrame([obj for res in logs for obj in res.to_json()])

FactoryResponse(function_name='foo',
                id='test@me.com',
                messages=[FactoryMessage(stage='hello_world',
                                         message='something happend',
                                         stage_num=None,
                                         is_success=True)],
                is_success=True)


FactoryResponse(function_name='foo',
                id='test2@me.com',
                messages=[FactoryMessage(stage='hello_world',
                                         message='something happend',
                                         stage_num=None,
                                         is_success=True)],
                is_success=True)




,function_name,stage,id,is_success,message,response
0,foo,hello_world,test@me.com,True,something happend,we did the thing
1,foo,hello_world,test2@me.com,True,something happend,we did the thing


## Factory Logs span across an entire implementation

In [8]:
# | exports


@dataclass
class FactoryLogs:
    """factory logs are the complete logs of an entire factory or script"""

    logs: List[FactoryResponse] = field(default_factory=lambda: [])

    def add_response(self, res: FactoryResponse):
        if res not in self.logs:
            self.logs.append(res)

        return self.logs

    def to_json(self) -> List[dict]:
        return [message for log in self.logs for message in log.to_json()]

#### sample process using Logs and a wrapper function

wrapper functions also implemented as  '@decorators' receive a function, `func`, as a parameter, and then execute code before and after `func` execution.

wrapper functions ease code development by implementing repeatable code within the wrapper and then altering how a base function, `func` appears to behave.

BEFORE function execution we might create an instance of `FactoryResponse`, res, and then pass that into `func`.  Note:  `func` must be prepared to receive `res` as an argument event if it isn't explicitly part of the end users' call.

AFTER function execution we can use tests to validate that `func` is behaving in a consistent way, or even raise exceptions if needed


In [9]:
def process_function_wrapper(func, id, logs):

    # pre function execution steps
    res = FactoryResponse(function_name=func.__name__, id=id)
    logs.add_response(res)

    # function execution
    result = func(res=res, id=id)

    # post processing and validation stpe
    if not isinstance(result, FactoryResponse):
        print("this isnt the right class")

    res.test_success()
    if not res.is_success:
        print("this wasnt successful")

    return result


def foo_condensed(res: FactoryResponse, id) -> FactoryResponse:
    res.add_message(
        FactoryMessage(stage="hello_world", message=f"retrieved {id}", is_success=True)
    )

    res.add_message(
        FactoryMessage(stage="next_layer", message=f"modified {id}", is_success=True)
    )

    # update response
    res.response = "we did the thing"
    res.test_success()

    return res


logs = FactoryLogs()  # will accumulate FactoryResponses for multiple foo executions

for id in ["test@me.com", "test2@me.com"]:  # run foo with each id
    process_function_wrapper(func=foo_condensed, logs=logs, id=id)

pd.DataFrame(logs.to_json())

,function_name,stage,id,is_success,message,response
0,foo_condensed,hello_world,test@me.com,True,retrieved test@me.com,we did the thing
1,foo_condensed,next_layer,test@me.com,True,modified test@me.com,we did the thing
2,foo_condensed,hello_world,test2@me.com,True,retrieved test2@me.com,we did the thing
3,foo_condensed,next_layer,test2@me.com,True,modified test2@me.com,we did the thing


In [11]:
#| export

class FactoryFunction_MissingParameter(Exception):
    def __init__(self, parameter, function_name):

        super().__init__(
            f"missing parameter - {parameter} - while calling {function_name}"
        )

@dataclass
class FactoryConfig:
    """create a dataclass which will have all the fields used in your config event"""

    auth: dec.Auth = field(repr=False)
    session: httpx.AsyncClient = field(repr=False)
    logs: FactoryLogs

    factory_fn_ls: List[Callable] = None

    def test_required_attr(self, attr_ls: List[str], function_name):
        for attr in attr_ls:
            if not getattr(self, attr):
                raise FactoryFunction_MissingParameter(attr, function_name)

    async def run(self, factory_fn_ls=None, debug_api: bool = False):

        factory_fn_ls = factory_fn_ls or self.factory_fn_ls

        for fn in factory_fn_ls:
            await fn(config=self, debug_api=debug_api, logs=self.logs)

        return self

In [12]:
# extend FactoryConfig with fields for this specific pipeline
@dataclass
class MyFactoryConfig(FactoryConfig):
    email: str = None
    token_name: str = "my_tokens"
    new_token = None  # will be updated by foo2


# use a simple wrapper to add pre and post process the main functions
# note we use id, config, and the wrapped function name to generate FactoryResponse
async def simple_wrapper(
    func: Callable,
    id,  # field from config to use to identify FactoryResponse
    logs,  # where we accumulate FactoryResponses
    config: FactoryConfig,
    debug_api: bool = False,
):

    res = FactoryResponse(function_name=func.__name__, id=getattr(config, id))

    logs.add_response(
        res
    )  # logs are mutable and passed by reference so will persist execution

    result = await func(res=res, config=config, debug_api=debug_api)

    # do some tests

    return result


async def foo1(
    res: FactoryResponse,
    config: FactoryConfig,
    debug_api: bool = False,
) -> FactoryResponse:

    # parameters are passed to message and APIs via config object
    res.add_message(
        FactoryMessage(
            stage="hello_world", message=f"retrieved {config.email}", is_success=True
        )
    )

    if debug_api:
        pass

    res.response = "we did the thing"
    res.test_success()
    return res


async def foo2(
    res: FactoryResponse,
    config: FactoryConfig,
    debug_api: bool = False,
) -> FactoryResponse:

    res.add_message(
        FactoryMessage(
            stage="generate_token",
            message=f"more work {config.token_name}",
            is_success=True,
        )
    )

    # config is mutable, so can be updated for downstream function use
    config.new_token = f"hello_{email}"

    res.response = "success"
    res.test_success()
    return res


# wrap foo functions to recycle code embedded within simple_wrapper
# add FactoryResponse to FactoryLog and pass res to foo functions
async def wrapped_foo1(logs, config, id="email", debug_api: bool = False):
    return await simple_wrapper(
        func=foo1, logs=logs, config=config, id=id, debug_api=debug_api
    )


async def wrapped_foo2(logs, config, id="email", debug_api: bool = False):
    return await simple_wrapper(
        func=foo2, logs=logs, config=config, id=id, debug_api=debug_api
    )


# ----- actual impelmentation ----
logs = FactoryLogs()

emails_to_process = ["test@me.com", "test@you.com"]

for email in emails_to_process:
    config = MyFactoryConfig(
        auth=123,
        email=email,
        session=httpx.AsyncClient(),
        logs=logs,
        factory_fn_ls=[
            wrapped_foo1,
            wrapped_foo2,
        ],  # this pipeline will execute these two functions sequentially
    )

    await config.run()

    # notice that config has new_token
    pprint(
        {
            "email": config.email,
            "new_token": config.new_token,
            "token_name": config.token_name,
        }
    )
    print("\n")

pd.DataFrame(logs.to_json())

{'email': 'test@me.com',
 'new_token': 'hello_test@me.com',
 'token_name': 'my_tokens'}


{'email': 'test@you.com',
 'new_token': 'hello_test@you.com',
 'token_name': 'my_tokens'}




,function_name,stage,id,is_success,message,response
0,foo1,hello_world,test@me.com,True,retrieved test@me.com,we did the thing
1,foo2,generate_token,test@me.com,True,more work my_tokens,success
2,foo1,hello_world,test@you.com,True,retrieved test@you.com,we did the thing
3,foo2,generate_token,test@you.com,True,more work my_tokens,success


# factory_function

use wrapper functions and decorators to enforce coding standards

### Factory_Function error classes

In [13]:
# | exports
class FactoryFunction_ResponseTypeError(TypeError):
    """a function wrapped in `process_function` must return FactoryResponse class"""
    def __init__(self, result):
        super().__init__(
            f"Expected function to return an instance of FactoryResponse. Got {type(result)} instead.  Refactor function to return FactoryResponse class"
        )
        
class FactoryFunction_Error(Exception):
    """base class for capturing errors within a factory function"""

    def __init__(
        self, factory: FactoryResponse, message: FactoryMessage, location=None
    ):

        e = f"💀 | {factory.function_name } | {factory.id} | {message.stage} - {message.message}"

        if location:
            e = f"{e} | in {location}"

        super().__init__(e)

 
class FactoryFunction_ResponseError(Exception):
    def __init__(self, message):
        super().__init__( message )

class FactoryFunction_NotSuccess(Exception):
    def __init__(self, messages : List[FactoryMessage]):
        super().__init__( "\n".join( [ message.to_string() for message in messages if not message.is_success ]) )


In [14]:
# | export

def factory_function(config_id_col,
                     **kwargs):
    def inner(func):

        @wraps(func)
        async def wrapper(*args,
                          config_id_col = config_id_col,
                          **kwargs): 
            
                                    
            if not kwargs.get('logs'):
                print(kwargs)
                raise FactoryFunction_MissingParameter( 'logs', func.__name__)
                                                       
            logs = kwargs['logs']
            
            res = FactoryResponse( function_name = func.__name__, id = getattr(kwargs['config'], config_id_col))
            logs.add_response(res)
            
            result = await func(*args, 
                                **kwargs,
                                res = res,
                                )
            
            
            if not isinstance(result, FactoryResponse):
                raise FactoryFunction_ResponseTypeError(result)
            
            if not result.response:
                raise FactoryFunction_ResponseError("factory response must return res.response")
            
            if not result.messages or len(result.messages) == 0:
                raise FactoryFunction_ResponseError("factory response must have messages")
            
            result.test_success()
                
            if not result.is_success:
                raise FactoryFunction_NotSuccess(result.messages)
                
            return result
        return wrapper
    return inner

In [15]:
# ---- setup ---- 

@dataclass
class MyFactoryConfig(FactoryConfig):
    email: str = None
    token_name: str = "my_tokens"
    new_token = None  # will be updated by foo2


@factory_function(config_id_col = 'email')
async def foo1(
    logs : FactoryLogs, # used by wrapper
    res: FactoryResponse, # passed in by wrapper
    config: FactoryConfig,
    debug_api: bool = False,
) -> FactoryResponse:

    # parameters are passed to message and APIs via config object
    res.add_message(
        FactoryMessage(
            stage="hello_world", message=f"retrieved {config.email}", is_success=True
        )
    )

    if debug_api:
        pass

    res.response = "we did the thing"
    res.test_success()
    return res

@factory_function(config_id_col = 'email')
async def foo2(
    logs : FactoryLogs, # used by wrapper
    res: FactoryResponse, # passed in by wrapper
    config: FactoryConfig,
    debug_api: bool = False,
) -> FactoryResponse:

    # parameters are passed to message and APIs via config object
    new_token = f"{config.token_name}_{config.email}"

    res.add_message(
        FactoryMessage(
            stage="regenerate_token", message='token revoked and regenerated', is_success=True
        )
    )

    if debug_api:
        pass

    res.response = new_token
    res.test_success()
    return res



logs = FactoryLogs()
emails_to_process = ["test@me.com", "test@you.com"]

for email in emails_to_process:
    config= MyFactoryConfig(auth = 123, email = email, 
                            session=httpx.AsyncClient(),
                            logs = logs,
                            factory_fn_ls = [foo1, foo2]
                            )
    await config.run()

pd.DataFrame(logs.to_json())

,function_name,stage,id,is_success,message,response
0,foo1,hello_world,test@me.com,True,retrieved test@me.com,we did the thing
1,foo2,regenerate_token,test@me.com,True,token revoked and regenerated,my_tokens_test@me.com
2,foo1,hello_world,test@you.com,True,retrieved test@you.com,we did the thing
3,foo2,regenerate_token,test@you.com,True,token revoked and regenerated,my_tokens_test@you.com


In [16]:
# | hide
import nbdev

nbdev.nbdev_export("./factory.ipynb")